<a href="https://colab.research.google.com/github/patshandofdoom/SECFindings/blob/main/SECScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference Data

[This is an edgar parser](https://github.com/rsljr/edgarParser). It can pull info from forms to show you what was written within




# Establish environment

In [1]:
#https://stackoverflow.com/questions/69025255/how-to-web-scraping-sec-edgar-10-k-dynamic-data
!pip --quiet install fake-useragent reticker lxml
from bs4 import BeautifulSoup
import requests
import sys
from fake_useragent import UserAgent
import xml.etree.ElementTree as ET
import re



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.8 MB/s eta 0:00:00


# Find New opportunities via the Benzinga live feed page


Open Benzinga web page, copy table and tuen into table

Need to adjust date in URL in order to narrow search.

https://www.benzinga.com/calendars/stock-splits?date_from=2023-04-27&date_sort=ex&date_to=2023-06-30

Need to store items that have already been sent using the "Ticker" and "Date Announced" in a seperate file. Any new item

In [2]:
##########################################################################################
#if day and month have <2 characters, add a 0
def adddigit(characters):
  if (len(characters) < 2):
    return( "0"+characters)
  else:
    return(characters)

#all required packages
from datetime import date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import requests
import re
#reticker makes it easier to regex tickers from a string
import reticker
extractor = reticker.TickerExtractor()
#type(extractor.pattern)


#get todays date
today = date.today()
startyear = str(today.year)
startmonth = adddigit(str(today.month))
startday = adddigit(str(today.day))

#get date 2 months out
enddate = date.today() + relativedelta(months=+2)
endyear = str(enddate.year)
endmonth = adddigit(str(enddate.month))
endday = adddigit(str(enddate.day))


base_url = f"https://www.benzinga.com/calendars/stock-splits?date_from={startyear}-{startmonth}-{startday}&date_sort=ex&date_to={endyear}-{endmonth}-{endday}"
theseheaders = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
print(base_url)
benzingaresponse = requests.get(base_url,headers=theseheaders)

benzingasoup = BeautifulSoup(benzingaresponse.text)

###############################################################################
#this segment checks live table on the benzinga website


###############################################################################
#this segment checks the live feed on the benzinga website
#find the div where updates are located
mydivs = benzingasoup.findAll('div',class_="mb-4")
#pull out all of the
allupdates = mydivs[0].findAll('a')

for updates in allupdates:
  #find all the spans in 'a' and make them into an array
  updatetitlespans = updates.findAll('span')

  #Sometimes the 'a' segments will have 3 spans, others will have more.
  #If it has 3 spans, we can grab the info from span 3 for the
  if(len(updatetitlespans)>3):
    stockticker = re.findall("\:(.*?)\)",updatetitlespans[2].text)
    instancetext = updatetitlespans[2].text
  #Otherwise, we grab the story text from span 2 and see if span 3 has any info about the ticker
  else:
    stockticker = re.findall("\:(.*?)\)",updatetitlespans[2].text)
    instancetext = updatetitlespans[1].text
  #If we cant find the ticker, try using extractor
  if len(stockticker) == 0:
    stockticker = extractor.extract(updatetitlespans[2].text)
  #If it still cant be found, make it N/A
  if len(stockticker) == 0:
    stockticker = "N/A"
  #finally, remove all space caharcters from the ticker
  else:
    stockticker = stockticker[0].replace(u'\xa0', '')
  #The split title will be in the second span
  instancetitle = updatetitlespans[1].text

  finalarray = [stockticker, instancetitle,instancetext]
  print(finalarray)

#next it need to keep a database of already sent items, then send the items to the discord bot
#from there, I wont have to look and refresh
#could possibly use ColabCTL to cron this script

https://www.benzinga.com/calendars/stock-splits?date_from=2023-06-29&date_sort=ex&date_to=2023-08-29
['ANY', 'Sphere 3D Announces 1-For-7 Share Consolidation; Will Begin Trading On A Post-Consolidation Basis On June 29, 2023', 'Sphere 3D Corp.\xa0(NASDAQ:ANY)\xa0("Sphere 3D" or the "Company"), dedicated to becoming the leading carbon-neutral Bitcoin mining company operating at an industrial scale, today announced that it']
['ALLR', 'Allarity Therapeutics Announces 1-For-40 Reverse Stock Split Of Common Stock; Will Begin Trading On A Post-Split Adjusted Basis On June 29, 2023', 'Allarity Therapeutics, Inc. (NASDAQ:ALLR) ("Allarity" or the "Company"), a clinical-stage pharmaceutical company developing novel oncology therapeutics together with drug-specific']
['ALLR', 'Reported June 27, Allarity Therapeutics Announces Clarification Of Effective Date For Reverse Stock Split Will Be Announced In Future Press Release', '\xa0Allarity Therapeutics, Inc. (NASDAQ:ALLR) ("Allarity" or the "Compan

# Finding the files that we need to search on SEC RSS Feed

In [ ]:
##########################################################################################
def getformlinkfromfilinglink(filinglink):
  headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

  #find the filing URL in the webpage
  edgar_resp = requests.get(filinglink,headers=headers)
  filingsoup = BeautifulSoup(edgar_resp.text)
  #First, get the table
  tableentries = filingsoup.findAll('table',class_='tableFile')

  #search through the rows for all data
  for eachrow in tableentries[0].findAll('tr'):
    celldata = eachrow.findAll('td')
    #As long as this row has tr labels and the type is an 8-k form, get the link to the 8-k
    if len(celldata)>0:
      if celldata[3].text=='8-K':
        for link in celldata[2].findAll('a',href=True):
          link= link['href'].split("ix?doc=")
          linktoform = "https://sec.gov"+link[1]
          return (linktoform)


#establish environment
import time
#get the url and establish headers
thisursl = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=8&company=&dateb=&owner=include&start=0&count=40&output=atom"
theseheaders = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
edgar_resp = requests.get(thisursl,headers=theseheaders)
thissoup = edgar_resp.text
#important to have a list of filings
formlist = []
root = ET.fromstring(thissoup)
#for all child elements check if it's an entry
for child in root:
  if child.tag =="{http://www.w3.org/2005/Atom}entry":
    #Then grab the title and href from each entry.
    for nestedchild in child:
      if nestedchild.tag == "{http://www.w3.org/2005/Atom}title":
        fulltitle = nestedchild.text
        formtype = fulltitle.split(" - ")
        formtype = formtype[0]
      if nestedchild.tag == "{http://www.w3.org/2005/Atom}link":
        formlink = nestedchild.attrib['href']
    childdict = {'formtype':formtype,'fulltitle':fulltitle, 'link':formlink}
    formlist.append(childdict)
print(formlist)

#need block here to open the filing page and get the right link from the table
for entry in formlist:
  if(entry['formtype']=='8-K'):
    print(entry['link'])
    formlink = getformlinkfromfilinglink(entry['link'])
    print(formlink)
    headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

    #find the filing URL in the webpage
    edgar_resp = requests.get(formlink,headers=headers)

    #Access the Filing
    formsoup = edgar_resp.text
    formsoup = formsoup.lower()

    #reverse count - count the number of times Reverse is in the text
    print(len(re.findall(r'reverse',formsoup,re.MULTILINE)))

    #merger count
    print(len(re.findall(r'merger',formsoup,re.MULTILINE)))
    time.sleep(1)

this section attempts to grab the form HTML from the filing page

In [ ]:
headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

#find the filing URL in the webpage
edgar_resp = requests.get('https://www.sec.gov/Archives/edgar/data/85535/000155837023010941/0001558370-23-010941-index.htm',headers=headers)
filingsoup = BeautifulSoup(edgar_resp.text)

tableentries = filingsoup.findAll('table',class_='tableFile')

#First, get the table
for eachrow in tableentries[0].findAll('tr'):
  celldata = eachrow.findAll('td')
  #As long as this row has tr signifiers and the type is an 8-k form, get the link to the 8-k
  if len(celldata)>0:
    if celldata[3].text=='8-K':
      for link in celldata[2].findAll('a',href=True):
        linktoform = "https://sec.gov"+link['href']
        print(linktoform)



sec.gov/ix?doc=/Archives/edgar/data/85535/000155837023010941/rgld-20230608x8k.htm


This code block will grab data from a filing URL and parse it for mentions of reverse or merger. Then count how many times each are mentioned. This will allow us to start parsing which filings we want to look at.

In [ ]:
#ua = UserAgent()
#headers = ua.random
headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

# Access page
cik = '200406'
type = '10-K'
dateb = '20210704'

# Obtain HTML for search page
#base_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={type}&dateb={dateb}"
base_url = 'ix?doc=/Archives/edgar/data/101984/000010198423000034/ueic-20230605.htm'

#have to split off the ix?doc bullcrap, get righ to the HTM link
link= base_url.split("ix?doc=")
base_url = 'https://www.sec.gov'+link[1]
edgar_resp = requests.get(base_url,headers=headers)

thissoup = edgar_resp.text
thissoup = thissoup.lower()

#reverse count - count the number of times Reverse is in the text
print(len(re.findall(r'reverse',thissoup,re.MULTILINE)))

#merger count
print(len(re.findall(r'universal',thissoup,re.MULTILINE)))


['', '/Archives/edgar/data/101984/000010198423000034/ueic-20230605.htm']
0
4


# Access Database


Need to figure out python routed database controls for planetscale database


In [3]:
!pip install --quiet python-dotenv mysqlclient mysql-connector-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 52.5 MB/s eta 0:00:00


In [4]:
#https://app.planetscale.com/prichardson0874/stock_live_updates
#https://planetscale.com/docs/tutorials/planetscale-quick-start-guide
#https://pynative.com/python-mysql-database-connection/


from dotenv import load_dotenv
import os
import MySQLdb
import mysql.connector
from mysql.connector import Error


load_dotenv('./.env')
print(os.getenv(HOST))

connection = mysql.connector.connect(
  host= os.getenv(HOST),
  user = os.getenv(USERNAME),
  passwd= os.getenv(PASSWORD),
  db = os.getenv(DATABASE),
  autocommit = True,
)
try:
  if connection.is_connected():
      db_Info = connection.get_server_info()
      print("Connected to MySQL Server version ", db_Info)
      cursor = connection.cursor()
      cursor.execute("select database();")
      record = cursor.fetchone()
      print("You're connected to database: ", record)

      ##this section is where the query code will go
      mySql_Create_Table_Query = """TABLE benzingaliveupdates(
                                  id_number int AUTO_INCREMENT,
                                  ticker varchar(8),
                                  title text NOT NULL,
                                  text text NOT NULL,
                                  split_amount int,
                                  date date,
                                  PRIMARY KEY(id_number))"""
      #""CREATE TABLE Laptop (Id int(11) NOT NULL, Name varchar(250) NOT NULL, Price float NOT NULL, Purchase_date Date NOT NULL, PRIMARY KEY (Id)) ""
      cursor = connection.cursor()
      result = cursor.execute(mySql_Create_Table_Query)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

NameError: ignored

# Access the Sheet for URLs
This segment will access the google sheet that I made in order to check each of the URL's for me. This should save a significant amount of time. I am still frustrated that I couldnt get this to work in google sheets natively, but I guess Python is just better in this way.

Importing data from google sheets
https://docs.gspread.org/en/v5.7.0/

The current work-around would be to use this segment first, then run the analysis segment with a sotred data table

In [ ]:
#https://docs.gspread.org/en/v5.7.2/user-guide.html#getting-a-cell-value
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#Open the stockwatch sheet
sh = gc.open_by_key('1gL6YuJdplpqQd3H4pHWZGqK6W7YDMkWkGmg3OswNcao')

#Open the Rev-split worksheet
worksheet = sh.worksheet('Rev-Spl Find')

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

Access and Write to Database

# Original Code

The code below is the original version of the fetch code that I was able to retrieve results



In [ ]:
#ua = UserAgent()
#headers = ua.random
headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

# Access page
cik = '200406'
type = '10-K'
dateb = '20210704'

# Obtain HTML for search page
#base_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={type}&dateb={dateb}"
base_url = 'https://www.sec.gov/Archives/edgar/data/2186/000165495423003704/bkti_8k.htm'
edgar_resp = requests.get(base_url,headers=headers)

print (edgar_resp)
edgar_str = edgar_resp.text

# Find the document link
soup = BeautifulSoup(edgar_str, 'html.parser')
print (soup)

s =  soup.find('span', recursive=True, string='SALES BY SEGMENT OF BUSINESS ')
t = s.find_next('table')
trs = t.find_all('tr')
for tr in trs:
    if tr.text:
        print(list(tr.stripped_strings))

<Response [200]>
<?xml version="1.0" encoding="utf-8"?><html xmlns="http://www.w3.org/1999/xhtml" xmlns:bkti="http://bktechnologies.com/20230323" xmlns:country="http://xbrl.sec.gov/country/2022" xmlns:currency="http://xbrl.sec.gov/currency/2022" xmlns:dei="http://xbrl.sec.gov/dei/2022" xmlns:dtr-types="http://www.xbrl.org/dtr/type/2020-01-21" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2015-02-26" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:nonnum="http://www.xbrl.org/dtr/type/non-numeric" xmlns:num="http://www.xbrl.org/dtr/type/numeric" xmlns:ref="http://www.xbrl.org/2006/ref" xmlns:srt="http://fasb.org/srt/2022" xmlns:srt-types="http://fasb.org/srt-types/2022" xmlns:us-gaap="http://fasb.org/us-gaap/2022" xmlns:us-roles="http://fasb.org/us-roles/2022" xmlns:utr="http://www.xbrl.org/2009/utr" xmlns:

AttributeError: ignored

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
worksheet = gc.open('Stock Watch').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)
# Go to https://sheets.google.com to see your new spreadsheet.